##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Como começar com o [TensorBoard.dev](https://tensorboard.dev)

O [TensorBoard.dev](https://tensorboard.dev) é um serviço gratuito e público do [TensorBoard](https://tensorflow.org/tensorboard) que permite fazer o upload e carregar seus experimentos de ML com todo mundo.

Este notebook treina um modelo simples e mostra como fazer upload de logs no TensorBoard.dev. [Prévia](https://tensorboard.dev/experiment/rldGbR8rRHeCEbkK61SWTQ).

### Configuração e importações

Este notebook usa os recursos do TensorBoard, que só estão disponíveis para versões &gt;= `2.3.0`.

In [ ]:
import tensorflow as tf
import datetime
from tensorboard.plugins.hparams import api as hp

### Treine um modelo simples e crie logs do TensorBoard

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

Os logs do TensorBoard são criados durante o treinamento ao passar as [callbacks do TensorBoard](https://www.tensorflow.org/tensorboard/get_started) e [de hiperparâmetros](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams) ao Model.fit() do Keras. Depois, esses logs podem ser carregados no TensorBoard.dev.


In [ ]:
model = create_model()
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1)
hparams_callback = hp.KerasCallback(log_dir, {
    'num_relu_units': 512,
    'dropout': 0.2
})

model.fit(
    x=x_train, 
    y=y_train, 
    epochs=5, 
    validation_data=(x_test, y_test), 
    callbacks=[tensorboard_callback, hparams_callback])

### (Somente Jupyter) Autorize o TensorBoard.dev

**Esta etapa não é necessária no Colab**

Nesta etapa, você precisa autorizar no console do seu shell, fora do Jupyter. No seu console, execute o comando a seguir.

`tensorboard dev list`

Como parte desse fluxo, você receberá um código de autorização. Esse código é necessário para consentir com os Termos de Serviço.

### Faça upload no TensorBoard.dev

O upload de logs do TensorBoard dará a você uma URL que pode ser compartilhada com todos.

Os TensorBoards carregados são públicos, então não envie dados sensíveis.

A pessoa responsável pelo upload saíra quando o logdir inteiro tiver sido carregado. (Isso é o que a flag `--one_shot` especifica.)

In [ ]:
!tensorboard dev upload --logdir ./logs \
  --name "Simple experiment with MNIST" \
  --description "Training results from https://colab.sandbox.google.com/github/tensorflow/tensorboard/blob/master/docs/tbdev_getting_started.ipynb" \
  --one_shot

Cada upload individual tem um ID de experimento único. Isso significa que, se você começar um novo upload com o mesmo diretório, obterá um novo ID de experimento. Você pode visualizar todos os experimentos carregados em https://tensorboard.dev/experiments/. Como alternativa, você pode listar seus experimentos no terminal usando o comando a seguir:

```
tensorboard dev list
```

In [ ]:
!tensorboard dev list

### Capturas de tela do TensorBoard.dev

Isto é o que aparecerá quando você acessar https://tensorboard.dev/experiments/:

![captura de tela da lista de experimentos do TensorBoard.dev](images/tbdev_experiment_list.png "TensorBoard.dev experiment list screenshot")

Isto é o que aparecerá ao acessar seu novo experimento no TensorBoard.dev:

![captura de tela do painel de controle de experimentos do TensorBoard.dev](images/tbdev_getting_started.png "TensorBoard.dev experiment dashboard screenshot")

### Exclua seu experimento do TensorBoard.dev

Para remover um experimento carregado, use o comando `delete` e especifique o `experiment_id` adequado. Na captura de tela acima, o experiment_id está listado no canto inferior esquerdo: `w1lkBAOrR4eH35Y7Lg1DQQ`.

In [ ]:
# You must replace YOUR_EXPERIMENT_ID with the value output from the previous
# tensorboard `list` command or `upload` command.  For example
# `tensorboard dev delete --experiment_id pQpJNh00RG2Lf1zOe9BrQA`

## !tensorboard dev delete --experiment_id YOUR_EXPERIMENT_ID_HERE